In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizerFast


2025-04-19 18:14:11.659323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745086451.848832      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745086451.911119      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [50]:
from datasets import load_dataset

# Load SQuAD (only 5K samples for speed)
dataset = load_dataset("squad", split="train[:11000]")

# Convert to pandas
df = dataset.to_pandas()


In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Combine all text for fitting (context + question)
all_texts = df['context'].tolist() + df['question'].tolist()

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(all_texts)


In [48]:
context_seqs = tokenizer.texts_to_sequences(df['context'].tolist())
question_seqs = tokenizer.texts_to_sequences(df['question'].tolist())


In [55]:
context_seqs[0]

[9918,
 2,
 104,
 28,
 7,
 1004,
 1068,
 1909,
 2,
 316,
 8863,
 1771,
 4880,
 11,
 7,
 1552,
 3268,
 3,
 2,
 4768,
 2131,
 1211,
 5,
 1416,
 3,
 2,
 316,
 253,
 4,
 2493,
 21,
 11,
 7,
 4645,
 3268,
 3,
 6403,
 15,
 3323,
 1,
 15,
 2,
 1498,
 1,
 3034,
 1075,
 1,
 432,
 6,
 2,
 316,
 253,
 11,
 2,
 4769,
 3,
 2,
 3999,
 1980,
 1211,
 1112,
 2,
 4769,
 11,
 2,
 5305,
 7,
 1,
 276,
 3,
 3666,
 4,
 1,
 21,
 11,
 7,
 1,
 3,
 2,
 5305,
 18,
 7392,
 532,
 89,
 2,
 4768,
 2131,
 1,
 716,
 6,
 1153,
 1,
 1,
 5,
 1,
 18,
 2,
 287,
 3,
 2,
 316,
 1936,
 4,
 5,
 7,
 1030,
 320,
 12,
 4159,
 114,
 146,
 8072,
 4,
 2,
 1771,
 4880,
 11,
 7,
 2706,
 293,
 1684,
 3268,
 3,
 2131]

In [52]:
from sklearn.model_selection import train_test_split

(
    context_train, context_test,
    question_train, question_test,
    ans_text_train, ans_text_test,
) = train_test_split(
    context_seqs, question_seqs, answer_texts,
    test_size=1000, random_state=42
)

In [53]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# max_context_len = max(len(seq) for seq in  context_seqs)
# print(max_context_len)
# max_question_len =  max(len(seq) for seq in  question_seqs)
# print(max_question_len)
max_context_len = 300
max_question_len = 30 
padded_contexts_train = pad_sequences(context_train, maxlen=max_context_len, padding='post', truncating='post')
padded_questions_train = pad_sequences(question_train, maxlen=max_question_len, padding='post', truncating='post')



In [70]:
padded_contexts_test = pad_sequences(context_test, maxlen=max_context_len, padding='post', truncating='post')
padded_questions_test = pad_sequences(question_test, maxlen=max_question_len, padding='post', truncating='post')

In [43]:
def word_to_token_id(word, tokenizer):
    """
    Returns the token index of a word using a fitted Keras Tokenizer.
    
    Parameters:
        word (str): The word to look up.
        tokenizer (Tokenizer): A Keras tokenizer that has already been fit.

    Returns:
        int: The token ID/index for the word, or None if not found.
    """
    word = word.lower()  # Keras Tokenizer lowercases by default
    return tokenizer.word_index.get(word, None)

In [45]:
token_id = word_to_token_id("in", tokenizer)
print("Token ID:", token_id)

Token ID: 5


In [7]:
max_context_len = max(len(seq) for seq in  padded_contexts)
maxxx=max(len(seq) for seq in  padded_questions)
print(max_context_len)
print(maxxx)

300
30


In [59]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

subset_df = df.iloc[:10000]

start_positions = []
end_positions = []

for idx, row in df.iterrows():
    context = row['context']
    answer_text = row['answers']['text'][0]
    answer_start_char = row['answers']['answer_start'][0]
    answer_end_char = answer_start_char + len(answer_text)

    # Tokenize context using Keras logic
    context_tokens = text_to_word_sequence(context)

    # Build character span mapping for each token
    char_to_token_map = []
    running_offset = 0
    for token in context_tokens:
        token_start = context.find(token, running_offset)
        token_end = token_start + len(token)
        char_to_token_map.append((token_start, token_end))
        running_offset = token_end

    # Initialize
    start_idx, end_idx = None, None

    # Find token indices for answer start and end
    for i, (start, end) in enumerate(char_to_token_map):
        if start_idx is None and start <= answer_start_char < end:
            start_idx = i
        if start <= answer_end_char <= end:
            end_idx = i
            break

    # Fallback: if end token not found, use start only
    if start_idx is None:
        start_idx = 0
    if end_idx is None:
        end_idx = start_idx

    start_positions.append(start_idx)
    end_positions.append(end_idx)


In [60]:
df['start_token'] = start_positions
df['end_token'] = end_positions


In [ ]:
context

In [58]:
start_positions

[0,
 32,
 0,
 0,
 14,
 0,
 70,
 0,
 21,
 142,
 0,
 0,
 0,
 0,
 0,
 69,
 7,
 0,
 42,
 0,
 0,
 13,
 0,
 0,
 61,
 0,
 0,
 0,
 0,
 8,
 0,
 48,
 0,
 0,
 24,
 0,
 32,
 0,
 0,
 17,
 25,
 38,
 86,
 108,
 6,
 59,
 91,
 0,
 140,
 0,
 57,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 8,
 1,
 0,
 173,
 183,
 15,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 109,
 132,
 13,
 33,
 232,
 255,
 9,
 1,
 24,
 58,
 5,
 62,
 0,
 27,
 48,
 0,
 36,
 0,
 15,
 0,
 61,
 0,
 39,
 0,
 0,
 12,
 0,
 0,
 37,
 63,
 46,
 51,
 51,
 94,
 0,
 136,
 0,
 0,
 0,
 0,
 52,
 0,
 69,
 0,
 0,
 4,
 0,
 0,
 30,
 0,
 14,
 82,
 15,
 0,
 0,
 0,
 0,
 0,
 52,
 0,
 96,
 33,
 1,
 29,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 126,
 0,
 15,
 0,
 50,
 0,
 17,
 0,
 62,
 0,
 0,
 23,
 71,
 0,
 0,
 29,
 0,
 0,
 31,
 5,
 14,
 21,
 0,
 67,
 0,
 10,
 20,
 0,
 42,
 36,
 0,
 29,
 0,
 0,
 57,
 0,
 86,
 104,
 1,
 0,
 33,
 0,
 76,
 0,
 0,
 0,
 49,
 56,
 11,
 0,
 0,
 56,
 0,
 21,
 45,
 0,
 81,
 0,
 3,
 0,
 0,
 0,
 0,
 9,
 58,
 0,
 79,
 69,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 31,
 62,
 0,
 0,
 0,
 0,
 0,
 

In [61]:
import numpy as np

# Initialize zero matrices for labels
y_start = np.zeros((len(df), max_context_len))
y_end = np.zeros((len(df), max_context_len))

# Loop over each sample and mark the correct token positions
for i, (s, e) in enumerate(zip(df['start_token'], df['end_token'])):
    if s < max_context_len:
        y_start[i, s] = 1
    if e < max_context_len:
        y_end[i, e] = 1


In [65]:
y_start_train = y_start[:10000]
y_end_train = y_end[:10000]


In [66]:
y_start_test= y_start[10000:11000]
y_end_test = y_end[10000:11000]


In [62]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Concatenate, RepeatVector
from tensorflow.keras.layers import TimeDistributed, Dropout
import tensorflow as tf
from tensorflow.keras.layers import Lambda


# Hyperparameters
embedding_dim = 100
hidden_size = 128
vocab_size = len(tokenizer.word_index) + 1  # from your tokenizer
max_context_len = 300
max_question_len = 30

# Inputs
context_input = Input(shape=(max_context_len,), name='context_input')
question_input = Input(shape=(max_question_len,), name='question_input')

# Embedding (you can load pretrained GloVe weights here)
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='embedding')

# Embedded sequences
context_embed = embedding_layer(context_input)     # (batch, context_len, embed_dim)
question_embed = embedding_layer(question_input)   # (batch, question_len, embed_dim)

# Bi-LSTM encoding
context_encoded = Bidirectional(LSTM(hidden_size, return_sequences=True))(context_embed)
question_encoded = Bidirectional(LSTM(hidden_size, return_sequences=False))(question_embed)

# Repeat question vector across context time steps
repeated_question = RepeatVector(max_context_len)(question_encoded)  # (batch, context_len, hidden*2)

# Concatenate context and question
merged = Concatenate()([context_encoded, repeated_question])  # (batch, context_len, hidden*4)

# Optional: Dropout for regularization
merged = Dropout(0.3)(merged)

# Output layers (start and end positions)
start_logits = TimeDistributed(Dense(1))(merged)  # (batch, context_len, 1)
start_logits = Lambda(lambda x: tf.squeeze(x, axis=-1))(start_logits)
start_probs = tf.keras.layers.Activation('softmax', name='start_output')(start_logits)

end_logits = TimeDistributed(Dense(1))(merged)
end_logits = Lambda(lambda x: tf.squeeze(x, axis=-1))(end_logits)
end_probs = tf.keras.layers.Activation('softmax', name='end_output')(end_logits)

# Final model
model = Model(inputs=[context_input, question_input], outputs=[start_probs, end_probs])
model.compile(
    optimizer='adam',
    loss={
        'start_output': 'categorical_crossentropy',
        'end_output': 'categorical_crossentropy'
    },
    metrics={
        'start_output': 'accuracy',
        'end_output': 'accuracy'
    }
)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ context_input             │ (None, 300)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ question_input            │ (None, 30)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 30, 100)        │      2,376,000 │ context_input[0][0],   │
│                           │                        │                │ question_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_3           │ (None, 256)            │        234,496 │ embedding[1][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 300, 256)       │        234,496 │ embedding[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ repeat_vector_1           │ (None, 300, 256)       │              0 │ bidirectional_3[0][0]  │
│ (RepeatVector)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 300, 512)       │              0 │ bidirectional_2[0][0], │
│ (Concatenate)             │                        │                │ repeat_vector_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 300, 512)       │              0 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_2        │ (None, 300, 1)         │            513 │ dropout_1[0][0]        │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_3        │ (None, 300, 1)         │            513 │ dropout_1[0][0]        │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 300)            │              0 │ time_distributed_2[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_3 (Lambda)         │ (None, 300)            │              0 │ time_distributed_3[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ start_output (Activation) │ (None, 300)            │              0 │ lambda_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ end_output (Activation)   │ (None, 300)            │              0 │ lambda_3[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,846,018 (10.86 MB)

 Trainable params: 2,846,018 (10.86 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
model.fit(
    [padded_contexts_train, padded_questions_train],   # inputs
    [y_start_train, y_end_train],                      # labels (two outputs)
    epochs=20,
    batch_size=32,
    validation_split=0.1,                  # optional: use 10% for validation
    verbose=1
)


Epoch 1/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 83s 154ms/step - end_output_accuracy: 0.3032 - end_output_loss: 4.2266 - loss: 8.1129 - start_output_accuracy: 0.3782 - start_output_loss: 3.8863 - val_end_output_accuracy: 0.3290 - val_end_output_loss: 4.0003 - val_loss: 7.7199 - val_start_output_accuracy: 0.4190 - val_start_output_loss: 3.6036
Epoch 2/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 37s 132ms/step - end_output_accuracy: 0.4129 - end_output_loss: 3.6423 - loss: 6.8717 - start_output_accuracy: 0.4961 - start_output_loss: 3.2294 - val_end_output_accuracy: 0.3290 - val_end_output_loss: 3.9857 - val_loss: 7.7157 - val_start_output_accuracy: 0.4190 - val_start_output_loss: 3.6059
Epoch 3/20
282/282 ━━━━━━━━━━━━━━━━━━━━ 37s 132ms/step - end_output_accuracy: 0.4160 - end_output_loss: 3.4860 - loss: 6.5369 - start_output_accuracy: 0.5045 - start_output_loss: 3.0509 - val_end_output_accuracy: 0.3290 - val_end_output_loss: 4.0427 - val_loss: 7.8300 - val_start_output_accuracy: 0.4180 - val_start_output_lo

In [71]:
start_preds, end_preds = model.predict([padded_contexts_test, padded_questions_test])

32/32 ━━━━━━━━━━━━━━━━━━━━ 15s 253ms/step


In [72]:
pred_start_indices = np.argmax(start_preds, axis=1)
pred_end_indices = np.argmax(end_preds, axis=1)

true_start_indices = np.argmax(y_start_test, axis=1)
true_end_indices = np.argmax(y_end_test, axis=1)

start_acc = np.mean(pred_start_indices == true_start_indices)
end_acc = np.mean(pred_end_indices == true_end_indices)

exact_match = np.mean((pred_start_indices == true_start_indices) & (pred_end_indices == true_end_indices))

print(f"Start Accuracy: {start_acc:.2%}")
print(f"End Accuracy: {end_acc:.2%}")
print(f"Exact Match: {exact_match:.2%}")

Start Accuracy: 31.80%
End Accuracy: 23.20%
Exact Match: 22.40%


In [40]:
answer_texts = [a['text'][0] for a in test_set['answers']]

In [80]:
context=df["context"]
questions=df["question"]


In [95]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# Assuming these are already sliced:
context = df["context"].iloc[10000:11000].tolist()
questions = df["question"].iloc[10000:11000].tolist()
answer_text = df['answers'].iloc[10000:11000].tolist()


for local_i in range(5):  # local index from 0 to 99
    tokens = text_to_word_sequence(context[local_i])
    print(context)
    print(question)
    print(answers)
    print(tokens)

    pred_ans = ' '.join(tokens[pred_start_indices[local_i]:pred_end_indices[local_i]+1])
    true_ans = ' '.join(tokens[true_start_indices[local_i]:true_end_indices[local_i]+1])
    raw_ans = answer_texts[local_i]

    print(f"Q: {questions[local_i]}")
    print(f"Predicted: {pred_ans}")
    print(f"Token-Mapped True: {true_ans}")
    print(f"Raw Answer from Dataset: {raw_ans}")
    print('-' * 50)

['in', 'turkey', 'and', 'the', 'ottoman', 'empire', 'the', 'oldest', 'technical', 'university', 'is', 'istanbul', 'technical', 'university', 'its', 'graduates', 'contributed', 'to', 'a', 'wide', 'variety', 'of', 'activities', 'in', 'scientific', 'research', 'and', 'development', 'in', '1950s', '2', 'technical', 'universities', 'were', 'opened', 'in', 'ankara', 'and', 'trabzon', 'in', 'recent', 'years', 'yildiz', 'university', 'is', 'reorganized', 'as', 'yildiz', 'technical', 'university', 'and', '2', 'institutes', 'of', 'technology', 'were', 'founded', 'in', 'kocaeli', 'and', 'izmir', 'in', '2010', 'another', 'technical', 'university', 'named', 'bursa', 'technical', 'university', 'was', 'founded', 'in', 'bursa', 'moreover', 'a', 'sixth', 'technical', 'university', 'is', 'about', 'to', 'be', 'opened', 'in', 'konya', 'named', 'konya', 'technical', 'university']
Q: When Konya Technical University opens, how many total institutes of technology will there be in Turkey and the Ottoman Empire

In [99]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

for local_i in range(20):

    tokens = text_to_word_sequence(context[local_i])

    pred_start = pred_start_indices[local_i]
    pred_end = pred_end_indices[local_i]
    true_start = true_start_indices[local_i]
    true_end = true_end_indices[local_i]

    if pred_start >= len(tokens) or pred_end >= len(tokens):
        pred_ans = "[out of range]"
    else:
        pred_ans = ' '.join(tokens[pred_start:pred_end+1])

    if true_start >= len(tokens) or true_end >= len(tokens):
        true_ans = "[out of range]"
    else:
        true_ans = ' '.join(tokens[true_start:true_end+1])

    # ✅ Raw answer from dataset (char span from original context)
    start_char = answer_starts[local_i]
    raw_ans = context[local_i][start_char:start_char + len(answer_texts[local_i])]

    # Print all
    print(f"\033[1mExample {local_i + 1}\033[0m")
    print(f"📘 Context:\n{context[local_i]}\n")
    print(f"❓ Question: {questions[local_i]}")
    print(f"✅ Raw Answer from Dataset (char slice): {raw_ans}")
    print(f"🎯 Token-Mapped True Answer: {true_ans}")
    print(f"🤖 Predicted Answer: {pred_ans}")
    print("-" * 100)

Example 1
📘 Context:
In Turkey and the Ottoman Empire, the oldest technical university is Istanbul Technical University. Its graduates contributed to a wide variety of activities in scientific research and development. In 1950s, 2 technical universities were opened in Ankara and Trabzon. In recent years, Yildiz University is reorganized as Yildiz Technical University and 2 institutes of technology were founded in Kocaeli and Izmir. In 2010, another technical university named Bursa Technical University was founded in Bursa. Moreover, a sixth technical university is about to be opened in Konya named Konya Technical University.

❓ Question: When Konya Technical University opens, how many total institutes of technology will there be in Turkey and the Ottoman Empire?
✅ Raw Answer from Dataset (char slice): r, a sixth technical unive
🎯 Token-Mapped True Answer: sixth
🤖 Predicted Answer: in
----------------------------------------------------------------------------------------------------
Ex

**Failed Attempt**

In [42]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

for i in range(1000):
    tokens = text_to_word_sequence(contexts[i])

    pred_ans = ' '.join(tokens[pred_start_indices[i]:pred_end_indices[i]+1])
    true_ans = ' '.join(tokens[true_start_indices[i]:true_end_indices[i]+1])
    raw_ans = answer_texts[i]  # ← Raw answer from the dataset

    print(f"Q: {questions[i]}")
    print(f"Predicted: {pred_ans}")
    print(f"Token-Mapped True: {true_ans}")
    print(f"Raw Answer from Dataset: {raw_ans}")
    print('-' * 50)

Q: When Konya Technical University opens, how many total institutes of technology will there be in Turkey and the Ottoman Empire?
Predicted: 
Token-Mapped True: sixth
Raw Answer from Dataset: six
--------------------------------------------------
Q: What two cities in Turkey acquired institutes of technology in the 1950s?
Predicted: 
Token-Mapped True: in
Raw Answer from Dataset: Ankara and Trabzon
--------------------------------------------------
Q: What year did UK polytechnics start functioning in a binary education system?
Predicted: 
Token-Mapped True: 1970
Raw Answer from Dataset: 1970
--------------------------------------------------
Q: What organization validates degrees received from polytechnic schools?
Predicted: 
Token-Mapped True: the
Raw Answer from Dataset: the UK Council for National Academic Awards
--------------------------------------------------
Q: What two-word term does Scotland use to describe their technological universities?
Predicted: 
Token-Mapped True: pol